# Project: Stochastic Gradient Descent
**Authors: Adam Wang, Hangxiao Zhu**

## Generate data

**Data Distribution D:** In practice, the data distribution is usually unknown. However, since you will be asked to generate training and test examples for the sake of running your experiments, we will describe a data distribution from which examples will be generated for each scenario. (Nevertheless, note that the SGD learner should remain oblivious to the distribution). Each example (x, y) is generated as follows:


*   with probability 1/2, set y = −1 and generate a (d−1)-dimensional Gaussian 
vector u ∼ N(μ0, σ2Id−1) where μ0 = (−1/4, −1/4, −1/4, −1/4) and Id−1 is the identity matrix of rank d − 1, that is, u is composed of 4 i.i.d. Gaussian components, each of mean −1/4 and variance σ2 (σ will be specified later).
*   with the remaining probability, set y = 1 and generate u ∼ N (μ1, σ2Id−1) where μ1 = (1/4, 1/4, 1/4, 1/4).

In [ ]:
import numpy as np

In [ ]:
def generate_data(size, sigma):
  data = np.zeros((4,size))
  labels = np.zeros((1,size))
  for x in range(0,size):
    if np.random.rand() > 0.5:
      data.T[x] = euclidean_projection(np.array([np.random.normal(-1/4, sigma),np.random.normal(-1/4, sigma),np.random.normal(-1/4, sigma),np.random.normal(-1/4, sigma)]))
      labels.T[x] = -1
    else:
      data.T[x] = euclidean_projection(np.array([np.random.normal(1/4, sigma),np.random.normal(1/4, sigma),np.random.normal(1/4, sigma),np.random.normal(1/4, sigma)]))
      labels.T[x] = 1
  data = np.vstack((np.ones((1,size)), data))
  return data, labels

Then, set x = ΠX(u) where ΠX is the Euclidean projection on to X,that is,u generated above is projected onto X (in case it lies outside X) and the resulting vector is denoted as x, which represents the feature vector.

In [ ]:
def euclidean_projection(u):
  size = u.size
  origin = np.zeros(size)
  distance = np.linalg.norm(u - origin)
  x = u
  if distance > 1:
    x = u / np.linalg.norm(u)
  return x

## SGD Implementation

In [ ]:
TESTING_SIZE = 400

In [ ]:
def logistic_loss(w, x, y): 
  return np.log(1 + np.exp(-y * np.dot(w, x)))

In [ ]:
def logistic_loss_gradient(w, x, y):
  return (-y * x * np.exp(-y * np.dot(w, x))) / (1 + np.exp(-y * np.dot(w, x)))

logistic_loss is convex and ||x||-Lipschitz

ρ = 5, M = ln(1+e^5)

In [ ]:
def SGD(data, labels, T, eta):
  w = np.zeros((T, 5))
  for t in range(0,T-1):
    i = np.random.randint(data.shape[1])
    G = logistic_loss_gradient(w[t], data.T[i], labels.T[i])
    w[t+1] = euclidean_projection((w[t].reshape(1,5) - eta*G))
  return np.mean(w, axis=0)

In [ ]:
def run(data, labels, T, eta, trials, training_size, sigma, testing_data, testing_labels):
  training_data, training_labels = generate_data(training_size, sigma)  
  w = np.zeros((trials, 5))
  mean_loss = np.zeros((trials, 1))
  std_loss = np.zeros((trials, 1))
  min_loss = np.zeros((trials, 1))
  classification_error = np.zeros((trials, 1))

  for i in range(0,trials):
    w[i] = SGD(data, labels, T, eta)
    traing_loss = logistic_loss(w[i], training_data, training_labels)
    testing_loss = logistic_loss(w[i], testing_data, testing_labels)
    mean_loss[i] = np.average(testing_loss)
    std_loss[i] = np.std(testing_loss)
    min_loss[i] = np.min(testing_loss)

    temp = np.sign(np.dot(w0, testing_data)) - testing_labels
    errors = np.where(temp == 0, temp, 1)
    classification_error[i] = np.average(errors)

  return np.mean(mean_loss), np.mean(std_loss), np.mean(min_loss), np.mean(std_loss-min_loss), np.mean(classification_error), np.std(classification_error)

### sigma=0.1, n=50

In [ ]:
testing_data, testing_labels = generate_data(TESTING_SIZE, 0.1)

In [ ]:
training_data, training_labels = generate_data(400, 0.1)

In [ ]:
w0 = SGD(training_data, training_labels, 10, 0.1)
w0

array([-0.04134415,  0.05170143,  0.05421373,  0.06070722,  0.05240799])

In [ ]:
testing_data, testing_labels = generate_data(50, 0.1)

In [ ]:
empirical_loss = logistic_loss(w0, training_data, training_labels)
np.average(empirical_loss)

0.6669848824726572

In [ ]:
testing_loss = logistic_loss(w0, testing_data, testing_labels)
np.average(testing_loss)

0.6683421971356942

In [ ]:
np.sign(np.dot(w0,testing_data))-testing_labels

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -2.,
         0., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])